In [2]:
install.packages("zoo")
library(zoo)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("data.table")
library(data.table)
require (data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘data.table’


The following objects are masked from ‘package:zoo’:

    yearmon, yearqtr




In [ ]:
unemp <- fread("UNRATE.csv")
print(unemp)

ERROR: Error in fread("../UNRATE.csv"): File '../UNRATE.csv' does not exist or is non-readable. getwd()=='/content'


In [ ]:
unemp [,DATE := as.Date (DATE) ]
a <- unemp [,DATE := as.Date (DATE) ]
print(a)

In [ ]:
setkey (unemp, DATE)

Генерирование набора данных co случайными пропущенными значениями

In [ ]:
rand.unemp.idx <- sample (1:nrow(unemp),length(nrow(unemp)))

In [ ]:
print(rand.unemp.idx)

In [ ]:
rand.unemp <- unemp[-rand.unemp.idx]

Генерирование набора данных с пропущенными значениями для высоких уровней безработицы.

In [ ]:
high.unemp.idx <- which (unemp$UNRATE > 8)

In [ ]:
high.unemp.idx

In [ ]:
num.to.select <- .2*length(high.unemp.idx)

In [ ]:
num.to.select

In [ ]:
high.unemp.idx <- sample(high.unemp.idx,)

In [ ]:
high.unemp.idx

In [ ]:
bias.unemp <- unemp[-high.unemp.idx]

In [ ]:
print(bias.unemp)

Чтобы создать набор данных с отсутствующими записями на основе таблиц данных с только что удаленными строками, нужно считать отсутствующие даты и значения NA, для чего лучше всего обратиться к инструментам скользящего объединения пакета data. table

In [ ]:
all.dates <- seq(from = unemp$DATE[1], to = tail(unemp$DATE, 1),
by = "months")
print(all.dates)

In [ ]:
rand.unemp = rand.unemp[J(all.dates), roll=0]
bias.unemp = bias.unemp[J(all.dates), roll=0]
rand.unemp[, rpt := is.na(UNRATE) ]

В скользящем объединении генерируется последовательность всех дат, которые должны оставаться доступными между начальной и конечной точками набора данных. Такой подход позволяет включать в набор данных строки с отсутствующими данными — обозначенные как NA.
Теперь, когда у нас есть наборы данных с отсутствующими значениями, рассмотрим несколько способов заполнения таких отсутствующих значений числами.

*   Замена предыдущим значением.
*   Скользящее среднее
*   Интерполяци

Будем сравнивать производительность обозначенных методов на наборах как со случайно отсутствующими, так и с систематически пропускаемыми данными. Поскольку оба набора данных получены из общего полного набора данных, нам будет намного проще проверить правдоподобность восполнения значений, избегая неправильных результатов. Конечно, решая реальные задачи, вы не будете располагать контрольным набором значений для проверки восполнения данных.

##Скользящее объединение
Скользящее объединение в data.table работает во многом подобно объединению данных в SQL, но только в масштабе временных рядов. Несмотря на то что таблицы далеко не всегда согласуются по заданным в них временным меткам, функция скользящего соединения обрабатывает их весьма точно.

  Фрагмент data.table с данными о датах взносов

In [ ]:
donations <- data.table(
amt = c(99, 100, 5, 15, 11, 1200),
dt = as.Date(c("2019-2-27", "2019-3-2", "2019-6-13",
               "2019-8-1", "2019-8-31", "2019-9-15"))
)

Информация о каждой информационной кампании

In [ ]:
publicity <- data.table(identifier = c("q4q42", "4299hj", "bbg2"), dt = as.Date(c("2019-1-1","2019-4-1","2019-7-1")))

Установка первичного ключа для каждого пакета data.table

In [ ]:
setkey (publicity, "dt")
setkey (donations, "dt")

Нужно снабдить данные о взносах в рамках последней кампании временными метками. Для этого параметру roll присваивается значение TRUE

In [ ]:
publicity[donations, roll = TRUE]

Теперь каждый взнос связывается с идентификатором, обозначающим информационную
кампанию, которая наиболее вероятно предшествовала благотворительным
взносам.

## Замена предыдущим значением
Один из самых простых способов восполнения пропущенных значений — заполнить их известными предыдущими значениями. Такой метод получил название *замена предыдущим значением*. Он основан на простом принципе и не требует дополнительных математических вычислений и сложных логических рассуждений. Просто представьте, что в ходе анализа имеющихся данных вы сдвигаете время на шаг вперед — недостающие значения оказываются заполненными зарегистрированными ранее данными. При таком подходе недостающие значения восполняются преимущественно предыдущими известными измерениями. Восполнение предыдущими значениями легко выполняется с помощью функции na.locf пакета zoo.

In [ ]:
rand.unemp[, impute.ff := na.locf(UNRATE, na.rm = FALSE)]
bias.unemp[, impute.ff := na.locf(UNRATE, na.rm = FALSE)]

Вывод графика упрощенного набора данных.

Результирующий график выглядит достаточно естественно, за исключением точек, в которых отсутствующие данные замещены повторяющимися значениями (рис. 6). Тем не менее, как показано на графике, значения, полученные подстановкой предыдущими данными, обычно сильно не отклоняются от истинных значений.

In [ ]:
unemp[350:400, plot (DATE, UNRATE, col = 1, lwd = 2, type = 'b')]
rand.unemp[350:400, lines(DATE, impute.ff, col = 2, lwd = 2, lty = 2)]
rand.unemp[350:400][rpt == TRUE, points(DATE, impute.ff,col = 2, pch = 6, cex = 2)]
rand.unemp[350:400, lines(UNRATE, impute.ff,col = 2, lwd = 2, lty = 2)]

Для более наглядного изучения полученного результата давайте построим сравнительные графики временных рядов, основанных на исходном и восполненном наборах значений. На них большинство значений должны совпадать абсолютно точно в силу общности их данных. На рис. 7 видно, что большинство точек данных полностью совпадает. На нем также показаны отдельные, разбросанные вокруг общей линии точки данных, но они не демонстрируют систематического смещения.

*Замена следующим значением*

Подобно замене недостающих данных предыдущими значениями для восполнения отсутствующих точек данных можно выбрать обратное направление течения времени и использовать в их качестве следующие известные значения. Тем не менее такой подход — это своего рода упреждение, поэтому он справедлив только в случаях отказа от использования полученных наборов данных в прогнозировании ситуаций. Такой способ восполнения обусловливается исключительно предметной областью и целесообразностью заполнения данных в обратном, а не прямом направлении.


In [ ]:
plot(unemp$UNRATE, rand.unemp$impute.ff)

Рис. 7. График истинного уровня безработицы и временного ряда с восполненными значениями. Как видите, замена отсутствующих данных предыдущими точками не вносит систематических искажений в исходный набор данных.

В некоторых ситуациях замена пропущенного значения предыдущим является наилучшим способом восполнения отсутствующих данных, даже при доступности других, более совершенных методов. Например, в медицинских задачах пропущенное значение часто указывает на то, что врач не счел необходимым провести дополнительные измерения, вероятно, посчитав состояние пациента вполне ожидаемым.
В подобных медицинских исследованиях замена отсутствующего значения последним известным будет вполне обоснованной, поскольку она базируется на убеждении медицинского работника в ненадобности пересмотра последних измерений.
Метод замены пропущенного значения предыдущим обладает целым рядом преимуществ: он применяется к последовательным потокам данных, не требует каких бы то ни было вычислений и не вносит систематических искажений в набор значений. Вскоре мы рассмотрим наглядный пример.

*Скользящее среднее*

Восполнить данные можно по методу скользящего среднего или медианы. Этот метод во многом подобен методу замены отсутствующих значений предыдущими.
В обоих случаях будущие значения предсказываются по известным предыдущим значениям (восполнение можно рассматривать как специальную форму предсказания). Однако метод скользящего среднего основан на данных нескольких последних временных точек.
Существует много ситуаций, когда скользящее среднее лучше подходит для целевой задачи, чем замена пропущенного значения предыдущим. Например, если данные зашумлены и у вас есть основания полагать, что значение отдельной пропущенной точки данных очень близко к среднему уровню значений, то для его определения следует использовать метод скользящего среднего, а не замены предыдущим значением. При замене данных предыдущими значениями в них вносится случайный шум, тогда как усреднение позволяет частично от него избавиться.
Чтобы избежать упреждения, усреднение нужно выполнять исключительно для временных точек, предшествующих пропущенному значению. Следовательно, метод можно реализовать следующим образом.


#####Скользящее среднее без упреждения

In [ ]:
rand.unemp[, impute.rm.nolookahead := rollapply(c(NA, NA, UNRATE), 3,
             function(x) {
                         if (!is.na (x[3])) x[3] else mean(x, na.rm = TRUE)
                        })]
bias.unemp[, impute.rm.nolookahead := rollapply(c(NA, NA, UNRATE), 3,
             function(x) {
                         if (!is.na (x[3])) x[3] else mean(x, na.rm = TRUE)
                        })]
print(rand.unemp)
print(bias.unemp)

Здесь отсутствующие значения представляются средними значениями всех предшествующих им значений (поскольку мы индексируем окончательное значение и используем это значение, чтобы определить, отсутствует ли оно и как его заменить). Скользящее среднее не обязательно рассчитывается через среднее арифметическое предыдущих данных. Например, экспоненциально взвешенное скользящее среднее позволяет придать больший вес последним значениям и меньший вес — более отдаленным. С другой стороны, среднее геометрическое лучше использовать во временных рядах, демонстрирующих сильную последовательную корреляцию или представленных последовательностью постоянно увеличивающихся значений.

При восполнении пропущенных данных методом скользящего среднего обязательно определитесь с тем, нужно ли принимать в расчет будущие значения, повышая риск возникновения упреждения. Если упреждение данным не грозит, то вычисление скользящего среднего лучше проводить как для прошлых, так и для будущих временных значений, максимально расширяя объем учитываемой информации и повышая точность прогнозирования значений. В подобных случаях нужно указывать диапазон или окно данных, рассматриваемых функцией rollapply() пакета zoo.

#####Скользящее среднее с упреждением

Как указывалось выше, вычисление скользящего среднего одновременно по данным прошлых и будущих временных точек прекрасно подходит при решении задач визуализации и сбора полных, непрерывных наборов данных, но оказывается несостоятельным при составлении прогнозов.
Результаты, полученные при вычислении скользящего среднего только по будущим значениям, а также расчета его одновременно по прошлым и будущим значениям, приведены на рис. 8.


In [ ]:
a <- rand.unemp[, complete.rm := rollapply(c(NA, UNRATE, NA), 3,
            function(x){
                      if (!is.na(x[2]))
                           x [2]
                      else
                          mean(x, na.rm = TRUE)
})]
print(a)
unemp[50:100, plot (DATE, UNRATE, col = 1, lwd = 2, type = 'b')]

Рис. 8. Точечной линией показан график временного ряда, восполненного по методу скользящего среднего без упреждения, а пунктирной линией — график временного ряда, данные которого восполнены по методу скользящего среднего с упреждением. Небольшими квадратиками обозначены восполненные без упреждения значения, а перевернутыми треугольниками — точки данных, полученные с упреждением. (необходимо дорисовать, что есть в описании)

Восполненный по методу скользящего среднего набор данных характеризуется низким уровнем дисперсии. Это необходимо учитывать при расчете точности вычислений, коэффициента детерминаций набора данных и других типов ошибок. Ваш расчет может привести к переоценке точности модели, что является частой проблемой, возникающей в ходе анализа временных рядов.

*Подбор набора данных для восполнения недостающих значений*

В процессе перекрестного анализа восполнение набора данных за счет включения в него отсутствующих точек, представленных средним или медианой, является общепринятой практикой. Хотя данный метод можно применять и по отношению к временным рядам, в большинстве случаев этого избегают. При вычислении среднего значения всего набора данных неизбежно учитывают будущие временные точки... а это упреждение!

*Интерполяция*

Интерполяция — это метод определения значений отсутствующих точек данных, основанный на геометрических ограничениях поведения данных. Например, в линейной интерполяции отсутствующие данные получаются в результате линейной аппроксимации известных соседних точек.
Наибольший интерес представляет линейная интерполяция, позволяя учитывать поведение системы во времени. Например, если известно, что система изменяется линейно во времени, то для аппроксимации отсутствующих данных лучше всего применять именно линейные зависимости. В байесовском понимании такой подход позволяет получать восполняемые значения с учетом априорных сведений о них.
Как и в случае скользящего среднего, интерполяция может основываться как на прошлых, так и на будущих значениях или учитывать только одно из направлений.
Остаются справедливыми стандартные предостережения: интерполяцию по будущим значениям стоит выполнять только в том случае, когда возникающее упреждение не оказывает заметного влияния на получаемый результат.
Ниже показано, как выполняется интерполяция данных с учетом прошлых и будущих временных точек (рис. 9).


#####Линейная интерполяция

In [ ]:
al<-rand.unemp[, impute.li := na.approx(UNRATE) ]
bl<-bias.unemp[, impute.li := na.approx(UNRATE) ]

#####Полиномиальная интерполяция

In [ ]:
ap <- rand.unemp[, impute.sp := na.spline (UNRATE) ]
bp <- bias.unemp[, impute.sp := na.spline (UNRATE) ]

In [ ]:
use.idx = 90:120
unemp [use.idx, plot(DATE, UNRATE, col = 1, type = 'b')]
rand.unemp[use.idx, lines(DATE, impute.li, col = 2, lwd = 2, lty = 2)]
rand.unemp[use.idx, lines(DATE, impute.sp, col = 3, lwd = 2, lty = 3)]

Рис. 9. Пунктирная линия соответствует линейной интерполяции, а точечная линия — сплайновой (полиномиальной)

Существует много ситуаций, в которых линейная (или сплайновая) интерполяция оказывается наиболее предпочтительным методом восполнения. Например, она будет справедливой в задачах исследования средней еженедельной температуры воздуха, имеющей тенденцию к повышению или понижению в зависимости от времени года, или ежегодных данных об объемах продаж быстро растущего предприятия. Если из года в год объем продаж предприятия увеличивается в одно и то же количество раз, то для восполнения отсутствующих данных лучше всего обратиться к линейной интерполяции. Иначе говоря, линейная интерполяция учитывает тенденции в данных, чего не скажешь об описанном выше методе скользящего среднего. В частности, если в данных просматривается тенденция к увеличению значения, то при использовании метода скользящего среднего восполняемые значения систематически недооценивались бы.
Однако учтите, что существует много ситуаций, в которых линейная (или сплайновая) интерполяция не находит достойного применения. Например, такому восполнению не подлежат пропущенные в наборе метеорологических данных сведения об осадках — изменение их количества происходит далеко не по линейному закону. Точно так при восполнении нескольких временных точек в наборе данных о суточном режиме сна нужно предельно настороженно отнестись к идее экстраполяции данных между двумя известными значениями. Например, одна из таких временных точек может относиться к бессонной ночи, а следующая — обозначать 30-минутную дремоту. Правильно оценить недостающие данные по ним будет крайне сложно.

*Сравнение методов*

Рассмотрев несколько различных методов восполнения, можно протестировать их на готовых наборах данных и сравнить полученные результаты.
Ранее мы создали два неполных набора данных, в одном из которых отсутствующие значения выбирались случайным образом, а в другом — в наиболее неудовлетворительных точках, соответствующих высокому уровню безработицы.
Сравнивая методы с позиции точности получаемых результатов, несложно заметить, что у них разные среднеквадратические ошибки.


In [ ]:
rand.unemp[, lapply(.SD,function(x){mean((x - unemp$UNRATE)^2,
na.rm = TRUE)}), .SDcols = c("impute.ff",
"impute.rm.nolookahead", "impute.li",
"impute.sp")]
bias.unemp [ , lapply(.SD, function (x) mean((x - unemp$UNRATE)^2, na.rm = TRUE)), .SDcols = c("impute.ff",
"impute.rm.nolookahead", "impute.li",
"impute.sp") ]

Помните, что во многих рассмотренных выше методах возникает упреждение. Единственные методы, в которых оно не наблюдается, — замены пропущенного значения предыдущим и скользящего среднего без упреждения (в отличие от скользящего среднего с упреждением). Совершенно неудивительно, что они обладают разными среднеквадратичными ошибками, а методы без упреждения не столь точны, как остальные.

*Последние замечания*

Выше рассмотрены простейшие и наиболее часто используемые методы восполнения отсутствующих данных временных рядов. Восполнение данных остается важной областью исследований дисциплины анализа данных. Чем важнее принимаемые решения, тем тщательнее нужно изучать возможные причины пропуска данных и последствия их восполнения. Ниже приведено несколько действенных советов по заполнению отсутствующих данных.

* Невозможно доказать, что в наборе отсутствуют действительно случайные данные, и тем более маловероятно, что их исключение из набора предопределяется естественными причинами.
* Иногда, но далеко не всегда, отсутствие отдельного измерения можно объяснить моделью. Широкие наборы данных со многими показателями — наилучшее средство для исследования закономерностей в пропущенных данных рассматриваемой модели. Тем не менее они не очень часто подвергаются анализу методами анализа временных рядов.
* Чтобы понять причины и масштабы неопределенности, вызванной восполнением набора данных отсутствующими значениями, необходимо проанализировать несколько сценариев и пообщаться с как можно большим количеством людей, вовлеченных в процесс сбора данных.
* Способ включения в набор недостающих данных должен учитывать их назначение.
Будьте предельно внимательны и постарайтесь избежать упреждения. В противном случае определите, насколько сильно упреждение влияет на точность получаемых результатов.

*Понижение и повышение частоты дискретизации*

Очень часто связанные временные ряды, полученные из разных источников данных, имеют неодинаковую частоту дискретизации. Это одна из многих причин, по которым вам придется изменять частоту дискретизации рядов данных, подвергаемых анализу. Разумеется, вы не можете изменить фактическую частоту сбора информации, но вам вполне по силам изменить частоту указания временных меток в исходном наборе. Такая операция называется понижением или повышением частоты дискретизации временных меток.
Нам уже доводилось сокращать временные данные в разделе “Получение набора временных рядов из таблиц”. Давайте еще раз рассмотрим эту задачу, но на этот раз предельно детально.
Понижение частоты дискретизации — это выбор подмножества данных, временные метки которых имеют меньшую частоту, чем в исходном временном ряду. Повышение частоты дискретизации позволяет получить наборы данных, которые якобы собирались чаще, чем на самом деле.

*Понижение частоты дискретизации*

Каждый раз, понижая частоту выборки данных, вы уменьшаете частоту дискретизации.
Чаще всего это делается в следующих случаях. Исходное разрешение данных слишком большое. Существует целый ряд причин, по которым высокая детализация данных не имеет смысла. Например, вы измеряете некий показатель чаще, чем того требует модель данных. Предположим, у вас есть набор данных о температуре воздуха, измеряемой каждую секунду. Исторический опыт (и здравый смысл) подсказывает, что измерения выполнялись слишком часто, и, скорее всего, набор данных включает слишком много повторяющейся информации, приводящей к дополнительной нагрузке на системы хранения и обработки данных. Более того, погрешность измерения может быть сопоставимой с посекундной вариацией температуры воздуха. В хранении чрезмерно больших и неинформативных наборов данных очень мало смысла. Уменьшение частоты дискретизации в таких наборах данных с регулярной выборкой сводится к выбору всех n-х элементов.
Выделение сезонных данных. Вместо того чтобы учитывать сезонность данных в общем временном ряду, образуйте частичный временной ряд, включающий данные только одного сезона. В следующем примере показан процесс понижения частоты дискретизации при образовании частичного временного ряда январских измерений, образованного из первоначального временного ряда ежемесячных данных. Как видите, теперь данные выбираются с годовой частотой.


In [ ]:
unemp[seq.int(from = 1, to = nrow(unemp), by = 12)]

Сравнение с более низкочастотными данными. Уменьшить частоту дискретизации данных может понадобиться при сопоставлении их с данными с заметной меньшей частотой выборки. В подобных случаях данные сначала подвергаются агрегации и только затем из них удаляются определенные временные точки. Агрегация может заключаться в такой простой операции, как усреднение или суммирование, или же представлять более сложное действие, подобное вычислению взвешенного среднего, в котором измеряемая величина характеризуется другими весовыми значениями в разных временных точках. Ранее в наборе данных о добровольных взносах нам уже доводилось выполнять понедельное суммирование значений, поскольку именно такая общая сумма взносов представляет интерес для дальнейшего анализа.
Напротив, в наборах экономических данных интерес будут представлять среднегодовые значения. В следующем примере мы будем рассчитывать среднюю величину вместо скользящей средней, подчеркивая актуальность веса среднеарифметического, а не последнего годичного значения (обратите внимание на различия в восполнении данных). В процессе группировки даты приводятся к строковому формату — обработка значений года является прекрасным примером использования SQL-подобных команд для обработки временных рядов.


In [ ]:
unemp[, mean (UNRATE), by = format(DATE, "%Y")]

*Повышение частоты дискретизации*

Повышение частоты выборки — это намного больше, чем операция, обратная понижению частоты дискретизации. Такое преобразование имеет отдельный смысл: в реальном мире уменьшение частоты измерений воспринимается вполне естественно, а вот повышение частоты выборки без увеличения количества измерений представляется безосновательной попыткой получения готового результата.
Процитирую автора популярного пакета XTS, предназначенного для обработки временных рядов, языка R (https://perma.cc/83E9-4N79).
Невозможно преобразовать ряд низкой периодичности в ряд более высокой периодичности — например, еженедельные данные в ежедневные или ежедневные данные в 5-минутные, — поскольку это потребует магии.
Однако существуют законные способы размечать данные с большей частотой, чем принято по умолчанию. Достаточно помнить об ограничениях, которые необходимо при этом учитывать. Помните, что вы всего лишь увеличиваете количество временных меток, а не пополняете ряд новыми данными.
Обсудим несколько ситуаций, когда повышение частоты дискретизации все же имеет здравый смысл.
Нерегулярные временные ряды. Одна из распространенных причин повышать частоту дискретизации заключается в необходимости получения регулярного ряда данных из нерегулярного временного ряда. Такое преобразование невозможно без увеличения частоты дискретизации, поскольку исходные данные конвертируются в данные, которые были собраны с заведомо большей частотой выборки. Если это ваш случай, то для повышения частоты дискретизации нужно воспользоваться функцией скользящего объединения языка R, как это делалось при заполнении отсутствующих значений экономического набора данных.


In [ ]:
all.dates <- seq(from = unemp$DATE[1], to = tail(unemp$DATE, 1),
by = "months")

Входные данные имеют разную частоту выборки. Иногда наряду с высокочастотными данными в модель нужно передать низкочастотную информацию, выровненную по частоте выборки исходных данных. Как и ранее, старайтесь избегать упреждения, но не забывайте, что, пока известные состояния остаются истинными и в данные не добавлены новые состояния, можно абсолютно безопасно увеличивать частоту выборки низкочастотных данных. Пусть нам абсолютно точно известно, что большинство вакантных рабочих мест появляется первого числа каждого месяца. Возникает соблазн использования значения месячного уровня безработицы, указанного в отчете, в качестве показателя всего месяца (это не будет упреждением, потому что уровень безработицы рассматривается неизменным в течение месяца).

In [ ]:
daily.unemployment = unemp[J(all.dates), roll = 31]
daily.unemployment

Понимание динамики изменения временных рядов. Располагая базовыми познаниями о временном поведении исследуемой величины, задачу увеличения частоты дискретизации можно свести к проблеме отсутствия данных. В этом случае к набору данных можно применять любые описанные ранее методы. Наиболее вероятным способом получения новых точек данных выступает интерполяция, но не забудьте убедиться в том, что выбранная модель данных полностью оправдывает такой выбор.
Как обсуждалось ранее, повышать или понижать частоту дискретизации может понадобиться даже в самом чистом наборе данных, поскольку эта операция выполняется почти всегда там, где приходится сравнивать показатели, изучаемые в разных временных масштабах. Примите к сведению, что пакет Pandas обладает невероятно удобной функцией увеличения и уменьшения частоты дискретизации — методом resample().

*Сглаживание данных*

Сглаживание данных выполняется по разным причинам, но чаще всего реальные временные ряды сглаживаются перед непосредственным анализом, особенно перед визуализацией, призванной наглядно проиллюстрировать историю их изменения. В этом разделе мы обсудим причины сглаживания данных, а также изучим наиболее распространенный метод сглаживания временных рядов — экспоненциальное сглаживание.

*Цель сглаживания*

Несмотря на то что обнаружение выбросов в данных является серьезной темой для рассмотрения, если у вас есть веские основания полагать, что данные требуют сглаживания, можете прибегнуть к методу скользящего среднего, чтобы исключить из набора резкие скачки показаний, ошибки измерения или и то, и другое вместе. Даже если скачки проявляются очень точно, они могут не относиться к измеряемой величине, а представлять ошибки измерительного оборудования.
В таких случаях данные нужно подвергнуть обязательному сглаживанию. Операция сглаживания данных тесно связана с задачей восполнения недостающих значений. Не удивительно, что в них обоих часто применяются одинаковые методы обработки данных. Например, вы можете сгладить данные по методу скользящего среднего как с упреждением, так и без него, — различие состоит только в выборе положения исследуемой точки относительно окна измерений, в котором рассчитывается сглаженное значение. Перед тем как приступить к сглаживанию данных, ответьте на ряд вопросов.
* Зачем нужно сглаживание? Сглаживание может служить ряду целей.

*Подготовка данных*

Необработанные данные не подходят для дальнейшей обработки? Например, вы можете знать, что слишком высокие значения маловероятны или нарушают определенные физические законы, но вам нужно обязательно учитывать их в дальнейшем анализе. Сглаживание — самое простое решение.

*Получение данных признаков*

На практике часто приходится строить выборки, извлекаемые из общих данных множественных характеристик людей, изображений или других объектов, которые включают значения всего нескольких признаков. Таким образом, полный набор данных сжимается до нескольких измерений, включая данные только отдельных характеристик. Генерация признаков особенно важна в машинном обучении.

*Прогнозирование*

Простейшей формой прогнозирования в некоторых процессах является операция возврата к среднему, которая основана на составлении прогнозов для сглаженных данных признаков.

*Визуализация*

Хотите выявить сигнал в том, что представляется зашумленной диаграммой рассеяния? Если хотите, то как вы собираетесь это сделать?
* Как сглаживание или отсутствие сглаживания влияет на получаемые результаты?

— Включаются ли в модель зашумленные и некоррелированные данные, и как сглаживание помогает бороться с ними?

— Необходимо ли сглаживать данные модели, описывающей реальное производство? Если да, то нужно воспользоваться методом сглаживания, который не приводит к упреждению.

— Рассматриваете ли вы некие общие принципы сглаживания или каждый раз подбираете метод по набору гиперпараметров? В последнем случае как удостовериться в том, что используемая форма перекрестной проверки не позволяет будущим данным оказывать влияние на получаемые результаты?

*Что такое гиперпараметры*

Термин гиперпараметры звучит устрашающе, но на практике это числа, которые используются для настройки или установки точности статистической модели или модели машинного обучения. Чтобы оптимизировать модель, необходимо протестировать ее на нескольких наборах гиперпараметров, которые часто подбираются по своего рода сетке значений. Поиск по сетке заключается в построении всех возможных комбинаций гиперпараметров в пространстве значений и проверке всех возможных комбинаций.
Например, представим, что изучаемый алгоритм принимает гиперпараметры А и Б, о которых известно, что разумное значение параметра А равно 0, 1 или 2, а у параметра Б оно представлено значением 0,7, 0,75 или 0,8. Тогда в нашем распоряжении находится сетка из девяти возможных комбинаций гиперпараметров — сочетание каждого возможного значения параметра А (их три) с каждым возможным значением параметра Б (их тоже три) предоставляет в общей сложности 3x3 = 9 возможностей. Как видите, настройка гиперпараметров может быть очень трудоемкой задачей.

##### Экспоненциальное сглаживание

При сглаживании данных обрабатывать все временные точки одинаково приходится крайне редко. В частности, относительно недавние данные могут быть более информативными, и в этом случае экспоненциальное сглаживание является наиболее приемлемым решением. В отличие от скользящего среднего, которое мы рассматривали ранее, где каждая точка, в которой отсутствовали данные, могла быть восполнена средним значением окружающих ее точек, экспоненциальное сглаживание более чувствительно ко времени и придает относительно недавним точкам больший вес, чем более отдаленным по времени. Таким образом, для данного окна измерений ближайший момент времени оказывается наиболее весомым, а вес точки, находящейся на большем временном удалении, уменьшается по экспоненциальному закону (отсюда и название метода).
Экспоненциальное сглаживание работает следующим образом. Для заданного периода времени t сглаженное значение ряда данных $S_t$ определяется по такой формуле:

$${{S}_{t}}=d\cdot {{S}_{t-1}}+\left( 1-d \right)\cdot {{x}_{t}}$$

Давайте представим, как изменяется ситуация с переходом к другой временной точке. Сглаженное значение в момент времени t - 1 рассчитывается по аналогичной формуле:

$${{S}_{t-1}}=d\cdot {{S}_{t-2}}+\left( 1-d \right)\cdot {{x}_{t-1}}$$

Подставив ее в исходную формулу, мы получим более сложное выражение вычисления сглаженного значения в момент времени t:

 $${{S}_{t}}=d\cdot \left( d\cdot {{S}_{t-2}}+\left( 1-d \right)\cdot {{x}_{t-1}} \right)+\left( 1-d \right)\cdot {{x}_{t}}$$

Легко распознать в этой записи ряд вида
$${{d}^{3}}\cdot {{x}_{t-3}}+{{d}^{2}}\cdot {{x}_{t-2}}+d\cdot {{x}_{t-1}}$$

Фактически именно благодаря такой форме записи экспоненциальное скользящее среднее считается легко разрешимым методом. Более подробная информация широко доступна в Интернете и приведена в многочисленных учебниках (см. издания, упомянутые в конце данного раздела).
Принципы сглаживания данных будут проиллюстрированы на примере, написанном на Python, поскольку его пакет Pandas включает один из самых широких наборов функций сглаживания. Функции сглаживания также широко применяются в языке R — они представлены как в стандартной библиотеке, так и в сторонних пакетах, специально предназначенных для обработки временных рядов.
Хотя рассматриваемые ранее примеры основывались на данных об уровнях безработицы в США, в текущем примере будет использован совершенно другой набор данных, описывающий пассажирские авиаперевозки (восходит к Боксу и широко известной книге Дженкинса о временных рядах). Исходный набор данных представляет собой отчет о количестве пассажиров (тысяч человек), ежемесячно перевозимых авиакомпанией, представленный с помесячной разбивкой.


In [ ]:
data(AirPassengers)
# <- fread('AirPassengers.csv')
stlRes <- stl(AirPassengers, s.window = "periodic")
#plot(stlRes)

In [ ]:
print(AirPassengers)
stlRes <- stl(AirPassengers, s.window = "periodic")
plot(stlRes)

Можно и по другому

In [ ]:
data_air<-read.csv("/content/AirPassengers.csv")
data_air_ts<-ts(data_air[,2], start=c(1949, 1), frequency=12)
print(data_air_ts)
decomposed <- decompose(data_air_ts)
plot(decomposed)

Рис. 12. Разложение исходного временного ряда на сезонную, трендовую и остаточную составляющие. Заметьте, что на осях у графиков откладываются совершенно разные величины. Также обратите внимание на серые столбцы у правого края каждого графика. Эти столбцы характеризуются одинаковым абсолютным размером (в единицах измерения соответствующих осей у) и указывают на различия в масштабировании отдельных составляющих